In [ ]:
import pyspark
from pyspark.sql import SparkSession, Row
# appName= "hive_pyspark"
# master= "local"
import matplotlib.pyplot as plt
from textblob import TextBlob
import numpy as np
from pyspark.sql.functions import col, asc,desc, avg
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()
from pyspark.sql.functions import udf,col, lower,greatest, monotonically_increasing_id
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel
from pyspark.mllib.tree import RandomForestModel, RandomForest
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.functions import vector_to_array
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, FloatType, StructType, StructField, StringType
from operator import itemgetter


In [ ]:
appName= "hive_pyspark"
master= "local"
spark = SparkSession.builder.master(master).appName(appName).enableHiveSupport().getOrCreate()
spark.sql("USE mmop_tesla_project")
df_stock = spark.sql("select * from stock")

In [ ]:
df_posts_all = spark.read.csv("gs://dataproc-staging-europe-west4-375495060785-ncrgfyir/notebooks/jupyter/df_sentiment.csv", header=True, inferSchema =True)

In [ ]:
def create_post_dataset(stock, X):
    
    start = stock["t"].min()
    stop = stock["t"].max()
    X_sub = X[(X['created_at'] <= stop) & (X['created_at'] >= start)]    
    X_sub["created_at"] = X["created_at"].astype(str)
    return X_sub


In [ ]:
# posts_data = df_posts_all.toPandas().loc[:, ["created_at", "sentiment"]]
# posts_data = posts_data.drop_duplicates()
# posts_data = posts_data.dropna().reset_index(drop=True)
# rows_to_drop = []
# for i in tqdm(range(len(posts_data))):
#     try:
#         posts_data.loc[i, "created_at"] = pd.to_datetime(posts_data.loc[i, "created_at"])
#     except:
#         rows_to_drop.append(i)
# posts_data = posts_data.drop(rows_to_drop, axis= 0)

In [ ]:
posts_data = pd.read_csv('gs://dataproc-staging-europe-west4-375495060785-ncrgfyir/notebooks/jupyter/post_data_to_stream.csv')
    

In [ ]:
stock_data = df_stock.groupby("t").agg(avg("p").alias("p")).orderBy(col("t").asc()) #.dropDuplicates(["t"])

stock_data = stock_data.toPandas()
stock_data["t"] = pd.to_datetime(stock_data["t"])
stock_data1 = stock_data.iloc[:1500, :]
window_size = 100
count = len(stock_data1)

posts_data["created_at"] = pd.to_datetime(posts_data["created_at"])

for i in tqdm(range(count //100)):
    stock_data_tmp = stock_data1.iloc[100*i: 100*i+100, :]  #spark.createDataFrame(stock_data1.tail(count-window_size*i)).limit(window_size)
    
    posts_data_tmp = create_post_dataset(stock_data_tmp, posts_data)    
    stock_data_tmp['num'] = range(1, len(stock_data_tmp) + 1)
    stock_data_tmp.to_csv(f'gs://dataproc-staging-europe-west4-375495060785-ncrgfyir/notebooks/jupyter/stream_data/stock/stock_file_{i}.csv', index = False)
    posts_data_tmp.to_csv(f'gs://dataproc-staging-europe-west4-375495060785-ncrgfyir/notebooks/jupyter/stream_data/posts/posts_file_{i}.csv', index = False)
    
